In [30]:
import numpy as np
import random 
import pandas as pd
import pyomo
import pyomo.environ as env

def create_random_distance_matrix(n):
# Creating a distance matrix that follows the triangle inequality
    matrix = np.zeros((n, n), dtype=int)

    for i in range(n):
        for j in range(i + 1, n):
            matrix[i, j] = random.randint(1, 100)

    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                matrix[j, k] = random.randint(1, 100)
                while matrix[i, j] + matrix[j, k] < matrix[i, k]:
                    matrix[i, k] = random.randint(1, 100)

    matrix = matrix + matrix.T
    return matrix

In [31]:
matrix = create_random_distance_matrix(10)

In [73]:
class Optimization:
    
    def __init__(self, cost_matrix, num_cities):
        self.cost_matrix = cost_matrix
        self.num_cities = num_cities
        
    def model(self):
        self.model = env.ConcreteModel()
        self.model.M = env.RangeSet(self.num_cities)  
        self.model.N = env.RangeSet(self.num_cities) 
        self.model.U = env.RangeSet(2, self.num_cities) 
        
        self.model.x = env.Var(self.model.N, self.model.M, within=env.Binary)
        
        self.model.u = env.Var(self.model.N, within=env.NonNegativeIntegers, bounds=(0, self.num_cities-1))
        self.model.c = env.Param(self.model.N, self.model.M, initialize=lambda model, i, j: self.cost_matrix[i-1][j-1])
        
        self.model.num_cities = self.num_cities  # Store num_cities in the model
        
        self.model.objective = env.Objective(rule=self.__obj_func, sense=env.minimize)
        self.model.const1 = env.Constraint(self.model.M, rule=self.__rule_const1)
        self.model.rest2 = env.Constraint(self.model.N, rule=self.__rule_const2)
        self.model.rest3 = env.Constraint(self.model.U, self.model.N, rule=self.__rule_const3)
        
    @staticmethod
    def __obj_func(model):
        return sum(model.x[i, j] * model.c[i, j] for i in model.N for j in model.M)
    
    @staticmethod
    def __rule_const1(model, M):
        return sum(model.x[i, M] for i in model.N if i != M) == 1
    
    @staticmethod
    def __rule_const2(model, N):
        return sum(model.x[N, j] for j in model.M if j != N) == 1
    
    @staticmethod
    def __rule_const3(model, i, j):
        if i != j: 
            return model.u[i] - model.u[j] + model.x[i, j] * model.num_cities <= model.num_cities - 1
        else:
            return model.u[i] - model.u[i] == 0 
    
    def print_model(self):
        self.model.pprint()
    
    def solve_model(self):
        solver = env.SolverFactory('glpk')
        result = solver.solve(self.model)
        print(result)
    
    def print_output(self):
        List = list(self.model.x.keys())
        valid_list = []
        for i in List:
            if self.model.x[i]() is not None and self.model.x[i]() != 0:
                valid_list.append(i)
        relation_dict = {item[0]: item[1] for item in valid_list}

        current = 1  # Start with the first element
        visited = set()  # Keep track of visited elements

        while current in relation_dict and current not in visited:
            print(current, end=' -> ')
            visited.add(current)
            current = relation_dict[current]
        print(1)


In [74]:
opt = Optimization(matrix, len(matrix))

In [75]:
opt.model()

In [76]:
opt.solve_model()


Problem: 
- Name: unknown
  Lower bound: 55.0
  Upper bound: 55.0
  Number of objectives: 1
  Number of constraints: 930
  Number of variables: 901
  Number of nonzeros: 4263
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 130349
      Number of created subproblems: 130349
  Error rc: 0
  Time: 286.15726137161255
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [77]:
opt.print_output()

1 -> 16 -> 27 -> 2 -> 22 -> 28 -> 18 -> 11 -> 21 -> 24 -> 19 -> 8 -> 30 -> 10 -> 26 -> 4 -> 17 -> 5 -> 20 -> 29 -> 6 -> 15 -> 3 -> 9 -> 12 -> 23 -> 14 -> 7 -> 25 -> 13 -> 1


In [59]:
class KOptimization:
    
    def __init__(self, cost_matrix, num_cities, num_drivers=3):
        self.cost_matrix = cost_matrix
        self.num_cities = num_cities
        self.num_drivers = num_drivers
        
    def model(self):
        self.model = env.ConcreteModel()
        self.model.M = env.RangeSet(self.num_cities)  
        self.model.N = env.RangeSet(self.num_cities) 
        self.model.U = env.RangeSet(2, self.num_cities) 
        
        self.model.x = env.Var(self.model.N, self.model.M, within=env.Binary)
        
        self.model.u = env.Var(self.model.N, within=env.NonNegativeIntegers, bounds=(0, self.num_cities-1))
        self.model.c = env.Param(self.model.N, self.model.M, initialize=lambda model, i, j: self.cost_matrix[i-1][j-1])
        
        self.model.num_cities = self.num_cities   # Store num_cities in the model
        self.model.num_drivers = self.num_drivers  # Store num_drivers in the model
        
        self.model.objective = env.Objective(rule=self.__obj_func, sense=env.minimize)
        self.model.rest1 = env.Constraint(self.model.M - [1], rule=self.__rule_const1)
        self.model.rest2 = env.Constraint(self.model.N - [1], rule=self.__rule_const2)
        self.model.rest4 = env.Constraint(self.model.N, rule=self.__rule_const4)
        self.model.rest5 = env.Constraint(self.model.M, rule=self.__rule_const5)
        self.model.rest3 = env.Constraint(self.model.U, self.model.N, rule=self.__rule_const3)
        
    @staticmethod
    def __obj_func(model):
        return sum(model.x[i, j] * model.c[i, j] for i in model.N for j in model.M)
    
    @staticmethod
    def __rule_const1(model, M):
        return sum(model.x[i, M] for i in model.N if i != M) == 1
    
    @staticmethod
    def __rule_const2(model, N):
        return sum(model.x[N, j] for j in model.M if j != N) == 1
    
    @staticmethod
    def __rule_const4(model, M):
        return sum(model.x[i, 1] for i in model.M if i != 1) == model.num_drivers

    @staticmethod
    def __rule_const5(model, N):
        return sum(model.x[1, j] for j in model.M if j != 1) == model.num_drivers


    @staticmethod
    def __rule_const3(model, i, j):
        if i != j: 
            return model.u[i] - model.u[j] + model.x[i, j] * model.num_cities <= model.num_cities - 1
        else:
            return model.u[i] - model.u[i] == 0 
    
    def print_model(self):
        self.model.pprint()
    
    def solve_model(self):
        solver = env.SolverFactory('glpk')
        result = solver.solve(self.model)
        print(result)
    
    def print_output(self):
        List = list(self.model.x.keys())
        valid_list = []
        for i in List:
            if self.model.x[i]() is not None and self.model.x[i]() != 0:
                valid_list.append(i)
        relation_dict = {item[0]: item[1] for item in valid_list}
        print(relation_dict)
        print(valid_list)

        current = 1  # Start with the first element
        visited = set()  # Keep track of visited elements

        while current in relation_dict and current not in visited:
            print(current, end=' -> ')
            visited.add(current)
            current = relation_dict[current]
        print(1)


In [60]:
opt = KOptimization(matrix, len(matrix))
opt.model()
opt.solve_model()
opt.print_output()



Problem: 
- Name: unknown
  Lower bound: 265.0
  Upper bound: 265.0
  Number of objectives: 1
  Number of constraints: 128
  Number of variables: 101
  Number of nonzeros: 585
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 25
      Number of created subproblems: 25
  Error rc: 0
  Time: 0.04130721092224121
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

{1: 10, 2: 7, 3: 5, 4: 1, 5: 1, 6: 8, 7: 6, 8: 3, 9: 2, 10: 1}
[(1, 4), (1, 9), (1, 10), (2, 7), (3, 5), (4, 1), (5, 1), (6, 8), (7, 6), (8, 3), (9, 2), (10, 1)]
1 -> 10 -> 1


In [61]:
l = [(1, 4), (1, 9), (1, 10), (2, 7), (3, 5), (4, 1), (5, 1), (6, 8), (7, 6), (8, 3), (9, 2), (10, 1)]

In [62]:
r = {item[0]: item[1] for item in l}

In [63]:
r

{1: 10, 2: 7, 3: 5, 4: 1, 5: 1, 6: 8, 7: 6, 8: 3, 9: 2, 10: 1}